In [1]:
import os
os.chdir(r'C:\Users\Abid\anaconda\envs')

In [3]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from timeseires.utils.to_split import to_split
from timeseires.utils.multivariate_multi_step import multivariate_multi_step
from timeseires.utils.multivariate_single_step import multivariate_single_step
from timeseires.utils.univariate_multi_step import univariate_multi_step
from timeseires.utils.univariate_single_step import univariate_single_step
from timeseires.utils.CosineAnnealingLRS import CosineAnnealingLRS
#from timeseires.callbacks.EpochCheckpoint import EpochCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint
# from timeseires.callbacks.TrainingMonitor import TrainingMonitor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Bidirectional, Add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D,TimeDistributed
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,MaxPooling1D,Concatenate,AveragePooling1D, GlobalMaxPooling1D, Input
from tensorflow.keras.models import Sequential,Model
import pandas as pd
import time, pickle
import numpy as np
import tensorflow.keras.backend as K
import tensorflow
from tensorflow.keras.layers import Input, Reshape, Lambda
from tensorflow.keras.layers import Layer, Flatten, LeakyReLU, concatenate, Dense
from tensorflow.keras.regularizers import l2
import glob
import h5py
import matplotlib.pyplot as plt
from keras.callbacks import Callback

In [4]:
#lookback = 24
model = None
start_epoch = 0
time_steps=24
num_features=21

In [5]:
def create_lstm():
    input_data = Input(shape=(time_steps, num_features))
    lstm_layer1 = LSTM(8, return_sequences=True)(input_data)
    lstm_layer2 = LSTM(20)(lstm_layer1)
    x = Flatten()(lstm_layer2)
    output_data = Dense(1)(x)
    model = Model(input_data, output_data)
    return model

In [6]:
model1 = create_lstm()
model1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 24, 21)]          0         
                                                                 
 lstm (LSTM)                 (None, 24, 8)             960       
                                                                 
 lstm_1 (LSTM)               (None, 20)                2320      
                                                                 
 flatten (Flatten)           (None, 20)                0         
                                                                 
 dense (Dense)               (None, 1)                 21        
                                                                 
Total params: 3,301
Trainable params: 3,301
Non-trainable params: 0
_________________________________________________________________


In [7]:
tensorflow.keras.utils.plot_model(model1 )

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [8]:
checkpoints = r'C:\Users\Abid\anaconda\envs\Lab_10\\E1-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
OUTPUT_PATH = r'C:\Users\Abid\anaconda\envs\Lab_10'
FIG_PATH = os.path.sep.join([OUTPUT_PATH,"\history.png"])
JSON_PATH = os.path.sep.join([OUTPUT_PATH,"\history.json"])

In [9]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
#TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1]

In [10]:
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model =create_lstm()
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] compiling model...


In [11]:
import os
path_dataset =r'C:\Users\Abid\anaconda\envs\Lab_10'
path_tr = os.path.join(path_dataset, 'AEP_train.csv')
df_tr = pd.read_csv(r'C:\Users\Abid\anaconda\envs\Lab_10\AEP_train.csv')
train_set = df_tr.iloc[:].values
path_v = os.path.join(path_dataset, 'AEP_validation.csv')
df_v = pd.read_csv(r'C:\Users\Abid\anaconda\envs\Lab_10\AEP_validation.csv')
validation_set = df_v.iloc[:].values 
path_te = os.path.join(path_dataset, 'AEP_test.csv')
df_te = pd.read_csv(r'C:\Users\Abid\anaconda\envs\Lab_10\AEP_test.csv')
test_set = df_te.iloc[:].values 

path_scaler = os.path.join(path_dataset, 'AEP_Scaler.pkl')
scaler      = pickle.load(open(path_scaler, 'rb'))

train_set.shape, validation_set.shape, test_set.shape

c:\Users\Abid\anaconda\envs\env\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


((84907, 21), (24259, 21), (12130, 21))

In [12]:
time_steps=24
num_features=21

In [13]:
start = time.time()
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=1)
validation_X, validation_y = univariate_multi_step(validation_set, time_steps, target_col=0,target_len=1)
test_X, test_y = univariate_multi_step(test_set, time_steps, target_col=0,target_len=1)
print('Time Consumed', time.time()-start, "sec")

Time Consumed 1.3561594486236572 sec


In [14]:
epochs = 6
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,verbose = verbose)

Epoch 1/6
2652/2653 [============================>.] - ETA: 0s - loss: 0.0461 - mae: 0.0461 - mape: 2373.8953
Epoch 1: val_loss improved from inf to 0.02133, saving model to C:\Users\Abid\anaconda\envs\Lab_10\E1-cp-0001-loss0.02.h5
2653/2653 [==============================] - 93s 31ms/step - loss: 0.0461 - mae: 0.0461 - mape: 2373.3933 - val_loss: 0.0213 - val_mae: 0.0213 - val_mape: 9.9637
Epoch 2/6
2652/2653 [============================>.] - ETA: 0s - loss: 0.0181 - mae: 0.0181 - mape: 482.6609
Epoch 2: val_loss improved from 0.02133 to 0.01298, saving model to C:\Users\Abid\anaconda\envs\Lab_10\E1-cp-0002-loss0.01.h5
2653/2653 [==============================] - 86s 32ms/step - loss: 0.0181 - mae: 0.0181 - mape: 482.5591 - val_loss: 0.0130 - val_mae: 0.0130 - val_mape: 6.7853
Epoch 3/6
2652/2653 [============================>.] - ETA: 0s - loss: 0.0125 - mae: 0.0125 - mape: 374.6430
Epoch 3: val_loss improved from 0.01298 to 0.01190, saving model to C:\Users\Abid\anaconda\envs\Lab_1

In [15]:
model = load_model(r'C:\Users\Abid\anaconda\envs\Lab_10\E1-cp-0006-loss0.01.h5', compile=False)

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 [==============================] - 5s 9ms/step
Mean Absolute Error (MAE): 151.02
Median Absolute Error (MedAE): 121.73
Mean Squared Error (MSE): 38555.95
Root Mean Squared Error (RMSE): 196.36
Mean Absolute Percentage Error (MAPE): 1.03 %
Median Absolute Percentage Error (MDAPE): 0.84 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)


In [16]:
checkpoints = r'C:\Users\Abid\anaconda\envs\Lab_10\E2-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
model=r'C:\Users\Abid\anaconda\envs\Lab_10\E1-cp-0033-loss0.01.h5'
start_epoch= 34

In [17]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

if model is None:
    print("[INFO] compiling model...")
    model = PC.build(time_steps=24, num_features=21, reg=0.0005)
    opt = Adam(1e-3)
    model.compile(loss='mae', optimizer=opt, metrics=["mae", "mape"])
else:
    print("[INFO] loading model from disk...")
    model = load_model(r'C:\Users\Abid\anaconda\envs\Lab_10\\E1-cp-0002-loss0.01.h5', compile=False)

    # Manually compile since we used compile=False
    opt = Adam(1e-3)
    model.compile(loss='mae', optimizer=opt, metrics=["mae", "mape"])

    # Print and update learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.learning_rate)))
    model.optimizer.learning_rate = 1e-4

    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.learning_rate)))


[INFO] loading model from disk...
[INFO] old learning rate: 0.0010000000474974513
[INFO] new learning rate: 9.999999747378752e-05


In [18]:
epochs = 5
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                        verbose = verbose)

Epoch 1/5
2653/2653 [==============================] - ETA: 0s - loss: 0.0118 - mae: 0.0118 - mape: 267.0082
Epoch 1: val_loss did not improve from 0.00983
2653/2653 [==============================] - 95s 32ms/step - loss: 0.0118 - mae: 0.0118 - mape: 267.0082 - val_loss: 0.0109 - val_mae: 0.0109 - val_mape: 5.3149
Epoch 2/5
2653/2653 [==============================] - ETA: 0s - loss: 0.0113 - mae: 0.0113 - mape: 279.9444
Epoch 2: val_loss did not improve from 0.00983
2653/2653 [==============================] - 82s 31ms/step - loss: 0.0113 - mae: 0.0113 - mape: 279.9444 - val_loss: 0.0106 - val_mae: 0.0106 - val_mape: 5.0043
Epoch 3/5
2653/2653 [==============================] - ETA: 0s - loss: 0.0111 - mae: 0.0111 - mape: 249.8033
Epoch 3: val_loss did not improve from 0.00983
2653/2653 [==============================] - 87s 33ms/step - loss: 0.0111 - mae: 0.0111 - mape: 249.8033 - val_loss: 0.0104 - val_mae: 0.0104 - val_mape: 4.8024
Epoch 4/5
2653/2653 [============================

In [19]:

model = load_model(r'C:\Users\Abid\anaconda\envs\Lab_10\\E1-cp-0002-loss0.01.h5', compile=False)

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 [==============================] - 6s 9ms/step
Mean Absolute Error (MAE): 207.59
Median Absolute Error (MedAE): 170.65
Mean Squared Error (MSE): 70070.06
Root Mean Squared Error (RMSE): 264.71
Mean Absolute Percentage Error (MAPE): 1.44 %
Median Absolute Percentage Error (MDAPE): 1.17 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)
